# Heart Attack Analysis and Prediction - Binary Classification with Logistic Regression

About dataset

* Age : Age of the patient
* Sex : Sex of the patient
* exang: exercise induced angina (1 = yes; 0 = no)
* ca: number of major vessels (0-3)
* cp : Chest Pain type chest pain type 
    * Value 1: typical angina
    * Value 2: atypical angina
    * Value 3: non-anginal pain
    * Value 4: asymptomatic
* trtbps : resting blood pressure (in mm Hg)
* chol : cholestoral in mg/dl fetched via BMI sensor
* fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
* rest_ecg : resting electrocardiographic results 
    * Value 0: normal
    * Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    * Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
* thalach : maximum heart rate achieved
* **target : 0= less chance of heart attack 1= more chance of heart attack**

### **This notebook proceeds with the steps below:)**
```
Step 1. Data Description
Step 2. EDA
Step 3. Correlation Check
Step 4. Test Data Split and Standard Scaling
Step 5. Modeling and Prediction (sklearn-LogisticRegression)
```

## Step 1. Data Description

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cd Heart_Attack_Analysis_and_Prediction

In [ ]:
ls

In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
import missingno as msno

In [ ]:
msno.bar(df);

In [ ]:
df.nunique()

In [ ]:
df.describe()

In [ ]:
df.shape

------------
1. All Columns are numeric
2. Null value isn't exist
3. colums -> x : 13, y : 1
4. rows -> 14
5. it need to be normalized

In [ ]:
df

# Step 2. EDA

In [ ]:
df.nunique()

----------
There are some categorical columns(but int type). Let's EDA that first

In [ ]:
df['sex'].value_counts()

In [ ]:
cat_cols = ['sex','cp','fbs','restecg','exng','slp','caa','thall']

In [ ]:
len(cat_cols)

In [ ]:
fig,  ax = plt.subplots(nrows=4, ncols=2, figsize=(20,16))
t = 0

for i in range(4):
    for j in range(2):
        cat_bar = [df[col].value_counts() for col in cat_cols]
        axes = ax[i][j]
        sns.barplot(x=cat_bar[t].index, y=cat_bar[t].values, ax=axes)
        axes.set_title(cat_cols[t])
        t += 1
plt.show()

In [ ]:
df.nunique()

-------
Let's see data info again - about categorical columns
* Sex : Sex of the patient
    * Value 0: Women
    * Value 1: Man
    
    
* exang: exercise induced angina 
    * Value 0: no (False)
    * Value 1: yes (True)
    
    
* ca: number of major vessels
    * Value 0: NOthing
    * Value 1: 1 vessel
    * Value 2: 2 vessel
    * Value 3: 3 vessel
    * Value 4: 4 vessel
    
    
* cp : Chest Pain type chest pain type 
    * Value 1: typical angina
    * Value 2: atypical angina
    * Value 3: non-anginal pain
    * Value 4: asymptomatic
    
    
* fbs : (fasting blood sugar > 120 mg/dl) 
    * Value 0: fasting blood sugar =< 120 (False)
    * Value 1: fasting blood sugar > 120 (True)
    
        
* rest_ecg : resting electrocardiographic results 
    * Value 0: normal
    * Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    * Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria


* **target : 0= less chance of heart attack 1= more chance of heart attack

-----------
- We don't have information about ['slp','thall','caa']
- We have information below (around)
    * Sex : Man - 70%
    * cp : Typical angina - 50%
    * fbs : fasting blood sugar <= 120(low) 85%
    * rest_ecg : 'Value 0 and 1' are 90% (normal or ST-T wave abnormal)
    * exang(exercise induced angina) : 0(False) 70%
    

In [ ]:
df.output.value_counts()

In [ ]:
output = df.output.value_counts()

In [ ]:
sns.barplot(x=output.index, y=output.values)

Well Classified 'Y(Output)'

# Step 3. Correlation Check

In [ ]:
df.info()

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(20,16))
sns.heatmap(df.corr())
plt.show()


In [ ]:
df.corr()

In [ ]:
df.corr()['output']

In [ ]:
df.corr()['output'].sort_values(ascending=False)

In [ ]:
Output = pd.DataFrame(df.corr()['output'].sort_values(ascending=False))

In [ ]:
sns.heatmap(Output)

------------
**There are high relation - Output and ['cp','thalachh','slp','restecg']**

# Step 4. Test Data Split and Standard Scaling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df

In [ ]:
train = df.iloc[:,:-1]

In [ ]:
test = df.iloc[:,-1:]

In [ ]:
train

In [ ]:
test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train,test, test_size=0.3, random_state=32)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

----------
Well split, rows=212, cols=(x:13, y:1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train_raw = scaler.fit_transform(X_train)
X_test_raw = scaler.transform(X_test)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
X_train = pd.DataFrame(X_train_raw, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_raw, columns=X_test.columns, index=X_test.index)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

-------------
OK, Well Scaled! Let's Build a Model

# Step 5. Modeling and Prediction

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train.values.ravel()) # ravel() : 1d - array transform
y_pred = lr.predict(X_test)


-----------------
!. Let's See more about ravel()

In [ ]:
y_train.values.ravel()

In [ ]:
y_train

This difference make important warning!

In [ ]:
(y_pred == y_test.values.ravel()).sum() / len(y_pred)

This code is same like ..

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_pred, y_test)